In [1]:
import numpy as np
import gdspy
import os
import cv2
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import shapely
import tqdm
import matplotlib.image
from geneticalgorithm import geneticalgorithm as ga
from slice_utilities import poly_intersection, slice_gds, cost_reward

In [3]:
in_file = '130nm_edit_0001.gds'
top_cell_name = 'top_io'
out_file='metal_slice.gds'
# bounding_box = [(493, 761), (493, 775), (700, 775), (700, 761)]
bounding_box = [(1200, 730),(1200, 775),(1390, 775),(1390,762), (1210, 762), (1210, 730)]
slice_gds(in_file,top_cell_name,bounding_box,out_file)

Layer: 108, Datatype: 0, Polygons: 59
Layer: 66, Datatype: 0, Polygons: 27
Layer: 61, Datatype: 0, Polygons: 169
Layer: 62, Datatype: 0, Polygons: 178
Layer: 63, Datatype: 0, Polygons: 155
Layer: 64, Datatype: 0, Polygons: 139
Layer: 65, Datatype: 0, Polygons: 67
Layer: 127, Datatype: 0, Polygons: 59
Layer: 30, Datatype: 0, Polygons: 57
Layer: 50, Datatype: 0, Polygons: 202
Layer: 43, Datatype: 0, Polygons: 83
Layer: 40, Datatype: 0, Polygons: 71
Layer: 10, Datatype: 0, Polygons: 150
Layer: 14, Datatype: 0, Polygons: 59
Layer: 74, Datatype: 0, Polygons: 66
Layer: 73, Datatype: 0, Polygons: 109
Layer: 72, Datatype: 0, Polygons: 304
Layer: 71, Datatype: 0, Polygons: 115
Layer: 70, Datatype: 0, Polygons: 106


In [ ]:
chip_r = gdspy.GdsLibrary(infile = out_file)

In [ ]:
chip_r = gdspy.GdsLibrary(infile='bb_slice.gds')


In [ ]:
chip_r.cells['(295, 0)'].get_paths()

Now that we have the reduced gds file we need to set some specific details to construct the needed masks

In [ ]:
chip_r = gdspy.GdsLibrary(infile='bb_slice.gds')
def test_poly(p,l,t):
    targ=[[493.000, 765.940],
    [493.000, 765.660],
    [651.890, 765.660],
    [651.890, 765.940],
    [493.000, 765.940]]
    poly=gdspy.Polygon(p-targ)
    if (poly.area()<0.01):
        return True
    else:
        return False
# changing layer 65 to remove the target polygon
targ=[[493.000, 765.940],
    [493.000, 765.660],
    [651.890, 765.660],
    [651.890, 765.940],
    [493.000, 765.940]]
cell = gdspy.Cell('target')
cell.add(gdspy.Polygon(targ, layer=500, datatype=500))
chip_r.cells['(65, 0)']
chip_r.cells['target']=cell
chip_r.cells['(65, 0)'].remove_polygons(test_poly)
gds_lib = gdspy.GdsLibrary()
gds_lib.write_gds(out_file[:-4]+'_t.gds', chip_r)

In [ ]:
chip_r.cells.keys()

Building the masks

In [ ]:
list_cost_keys = list(chip_r.cells.keys())
list_cost_keys.remove("top")
list_cost_keys.remove("(127, 0)")
list_cost_keys.remove("target")
print(list_cost_keys)
list_reward_keys=list()
list_reward_keys.append("target")

section_bb = Polygon([(493, 682), (493, 857), (788, 857), (788, 682)])  # Replace with your section coordinates
points = section_bb.exterior.xy
min_x, max_x = int(min(points[0])),int(max(points[0]))
min_y, max_y = int(min(points[1])),int(max(points[1]))
w = int(max_x - min_x)
h = int(max_y - min_y)

reselution = 100
mask_cost = np.zeros((w*reselution,h*reselution))
mask_reward = np.zeros((w*reselution,h*reselution))

for t in list_cost_keys:
    polygons = chip_r.cells[t].get_polygons()
    for p in polygons:
        ver=[]
        for i in range(len(p)):
            ver.append([reselution*(p[i,1]-min_y), reselution*(p[i,0]-min_x)])
        ver = np.array(ver,dtype=int)
        cv2.fillPoly(mask_cost, pts=[ver], color=(255,0,0))


for t in list_reward_keys:
    polygons = chip_r.cells[t].get_polygons()
    for p in polygons:
        ver=[]
        for i in range(len(p)):
            ver.append([reselution*(p[i,1]-min_y), reselution*(p[i,0]-min_x)])
        ver = np.array(ver,dtype=int)
        cv2.fillPoly(mask_reward, pts=[ver], color=(255,0,0))

